In [202]:
# 最高ウェイト表現のウェイトを最高ウェイトとの差で表す関数。
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [203]:
# Parabolic subgroupを表すクラス
class ParabolicSubgroup:
    def __init__(self, G, L, crossed_out_nodes) -> None:
        self.G = G
        self.L = L
        self.crossed_out_nodes = crossed_out_nodes
        self.R_G = WeylCharacterRing(self.G)
        self.R_L = WeylCharacterRing(self.L)

    def __repr__(self) -> str:
        return f'the parabolic subgroup of {self.G} with crossed-out nodes {self.crossed_out_nodes}'

    def dynkin_diagram(self):
        return self.G.marked_nodes(self.crossed_out_nodes).dynkin_diagram()

    def ambient_space(self):
        return self.R_G.space()
    
    def ambient_space_dimension(self):
        return self.R_G.space().dimension()

    # Sagemathのライブラリでは1-indexを用いるが、ここでは簡単のためとりあえず0-indexで表す
    def simple_roots(self):
        return [ self.R_G.simple_roots()[i] for i in set(1..self.G.rank()) - set(self.crossed_out_nodes)]

    def positive_roots(self):
        L = self.ambient_space()

        # positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
        roots = [ pr for pr in self.G.root_system().root_lattice().positive_roots()]
        for i in self.crossed_out_nodes:
            roots = [ pr for pr in roots if not (pr - (self.G.root_system().root_lattice().simple_roots())[i]).is_positive_root()]

        # ambient spaceの元に変換
        return [ L(pr) for pr in roots ]

    # 引数の`weight`は基本ウェイトを基底にして表示したもの
    def weight_muliplicities(self, weight) -> dict:
        # GとLのディンキン図の頂点のずれを補正する関数
        def correct_index(index: int) -> int:
            for i in range(len(self.crossed_out_nodes)):
                if index + i < self.crossed_out_nodes[i]:
                    return index + i
            return index + len(self.crossed_out_nodes)

        fws_L = [fw for fw in self.R_L.fundamental_weights()] # conversion from 1-index to 0-index
        weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(self.crossed_out_nodes)] 
        weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(self.L.rank()))

        fws_G = [fw for fw in self.R_G.fundamental_weights()]
        weight_for_G = sum( weight[i] * fws_G[i] for i in range(self.G.rank()))

        mul_set = root_difference_multiplicities(self.R_L, weight_for_L)

        result = {}
        for k, v in mul_set.items():
            w = weight_for_G \
                + sum(k[i - 1] * self.R_G.simple_roots()[correct_index(i)] 
                    for i in (1..self.L.rank()))
            result[w] = v

        return result
    
    def weyl_dimension(self, weight) -> int:
        return self.ambient_space().weyl_dimension(sum(weight[i - 1] * self.R_G.fundamental_weights()[i] for i in (1..self.G.rank())))

In [204]:
P = ParabolicSubgroup(CartanType('A3'), CartanType('A2'), [1])
P

the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [205]:
P.dynkin_diagram()

X---O---O
1   2   3   
A3 with node 1 marked

In [206]:
P.ambient_space_dimension()

4

In [207]:
P.simple_roots()

[(0, 1, -1, 0), (0, 0, 1, -1)]

In [208]:
P.positive_roots()

[(0, 1, -1, 0), (0, 0, 1, -1), (0, 1, 0, -1)]

In [209]:
P.weight_muliplicities((1, 2, 0))

{(3, 2, 0, 0): 1,
 (3, 1, 1, 0): 1,
 (3, 0, 2, 0): 1,
 (3, 1, 0, 1): 1,
 (3, 0, 1, 1): 1,
 (3, 0, 0, 2): 1}

In [332]:
from abc import ABC, abstractmethod

class IVectorBundle(ABC):
    pass

class IVariety(ABC):
    @abstractmethod
    def dimension(self) -> int:
        pass

    @abstractmethod
    def tangent_bundle(self) -> IVectorBundle:
        pass

    def cotangent_bundle(self) -> IVectorBundle:
        return self.tangent_bundle().dual()

    @abstractmethod
    def integration(self, f) -> int:
        pass

    def chern_classes(self) -> list:
        return self.tangent_bundle().chern_classes()

    def todd_classes(self) -> list:
        return self.tangent_bundle().todd_classes()

In [345]:
import re
singular.lib("chern.lib")


class IVectorBundle(ABC):
    @abstractmethod
    def base(self) -> IVariety:
        pass

    @abstractmethod
    def rank(self) -> int:
        pass

    @abstractmethod
    def chern_classes(self) -> list:
        pass

    # 演算子のオーバーロード
    def __add__(self, other) -> IVectorBundle:
        return direct_sum(self, other)

    def __mul__(self, other) -> IVectorBundle:
        return tensor_product(self, other)

    def chern_character(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))
        
        # Using Singular, calclate universal formula of chern character
        singular.lib('chern.lib')
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        ch_str_list = singular.chAll(l, self.base().dimension()).sage_structured_str_list()
        chern_character = [ ring_for_ch(self.rank()) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ chern_character[i](chern_classes) for i in (0..self.base().dimension()) ]

    def todd_classes(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

        # Using Singular, calculate universal formula of Todd classes
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        todd_str_list = singular.todd(l).sage_structured_str_list()
        todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ todd_classes[i](chern_classes) for i in (0..self.base().dimension()) ]

    def dual(self) -> IVectorBundle:
        vector_bundle = self

        class VB(IVectorBundle):
            def rank(self) -> int:
                return vector_bundle.rank()
            
            def base(self) -> IVariety:
                return vector_bundle.base()

            def chern_classes(self) -> list:
                chern_classes = vector_bundle.chern_classes()
                return [ (-1)^i * chern_classes[i] for i in range(len(chern_classes))]

            def __repr__(self) -> str:
                return f'the dual vector bundle of {vector_bundle}'

        return VB()

In [212]:
def direct_sum(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    
    rank = vector_bundle1.rank() + vector_bundle2.rank()
    base = vector_bundle1.base()

    cc = sum(c1 for c1 in vector_bundle1.chern_classes()) * sum(c2 for c2 in vector_bundle2.chern_classes())
    chern_classes = [
        homogeneous_part(cc, i)
        for i in (0..vector_bundle1.base().dimension())
    ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return chern_classes

        def __repr__(self) -> str:
            return f'the direct sum of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [213]:
def tensor_product(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    rank1 = vector_bundle1.rank()
    rank2 = vector_bundle2.rank()

    len_cc1 = len(vector_bundle1.chern_classes()) - 1
    len_cc2 = len(vector_bundle2.chern_classes()) - 1
    
    ring_for_Es = PolynomialRing(QQ, [ f'c{i}_E1' for i in (1..len_cc1) ] + [ f'c{i}_E2' for i in (1..len_cc2) ])
    

    r = singular.ring(0, f'(c(1..{len_cc1}), C(1..{len_cc2}))', 'dp')
    l1 = singular.list(f'c(1..{len_cc1})')
    l2 = singular.list(f'C(1..{len_cc2})')

    ch_str_list = singular.chProd(rank1, l1, rank2, l2).sage_structured_str_list()
    ch_prod = [ re.sub(r'c\(([0-9]+)\)', r'c\1_E1', s) for s in ch_str_list[:vector_bundle1.base().dimension()] ]
    ch_prod = [ ring_for_Es(re.sub(r'C\(([0-9]+)\)', r'c\1_E2', s)) for s in ch_prod ]


    cc = [ 1 ] + [ cp(vector_bundle1.chern_classes()[1:] + vector_bundle2.chern_classes()[1:]) for cp in ch_prod ]
    

    rank = vector_bundle1.rank() * vector_bundle2.rank()
    base = vector_bundle1.base()


    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the tensor product of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [214]:
# `degree`次部分を取り出す関数
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [215]:
def chern_number(variety: IVariety, degrees: list) -> int:
    if sum(d for d in degrees) != variety.dimension():
        return 0
    else:
        chern_classes = variety.chern_classes()
        return variety.integration(
            prod([chern_classes[d] for d in degrees])
        )

In [216]:
class FlagVariety(IVariety):
    def __init__(self, parabolic_subgroup) -> None:
        self.parabolic_subgroup = parabolic_subgroup

        # コホモロジー環を含む環
        self.ring = PolynomialRing(QQ, 'x', parabolic_subgroup.ambient_space_dimension()) 
        self.x = self.ring.gens()

        self.tangent_weights = [
            sum( r[l] * self.x[l] for l in range(parabolic_subgroup.ambient_space_dimension())) 
            for r in set(parabolic_subgroup.R_G.positive_roots())-set(parabolic_subgroup.positive_roots())
        ]
        self.dim = len(self.tangent_weights)

    def __repr__(self) -> str:
        return f'a flag_variety associated to {self.parabolic_subgroup}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        tangent_weights = { w: 1 for w in set(self.parabolic_subgroup.R_G.positive_roots())-set(self.parabolic_subgroup.positive_roots()) }
        return EquivariantVectorBundle(self, tangent_weights)


    # 次数ごとのchern類
    def chern_classes(self):
        return [ 
            homogeneous_part(prod(1+x for x in self.tangent_weights), i)
            for i in (0..self.dim) 
        ]

    def numerical_integration_by_localization(self, f):
        random_x = [RealField(1000)(random()) for i in range(self.parabolic_subgroup.ambient_space_dimension())]
        orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(self.parabolic_subgroup.G)]
        top_of_f = homogeneous_part(f, self.dim)
        denominator_in_localization = prod(self.tangent_weights)
        if top_of_f == 0:
            return 0
        else:
            return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(self.parabolic_subgroup.L))

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)
     

In [217]:
class EquivariantVectorBundle(IVectorBundle):
    def __init__(self, flag_variety, weight_muliplicities) -> None:
        self.flag_variety = flag_variety
        self.weight_muliplicities = weight_muliplicities
        self.rk = sum(v for v in self.weight_muliplicities.values())

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight_muliplicities}'

    def rank(self) -> int:
        return self.rk

    def base(self) -> FlagVariety:
        return self.flag_variety

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        cc = prod((1 + class_from_weight(vector(w)))^i for w, i in self.weight_muliplicities.items())

        return [
            homogeneous_part(cc, i)
            for i in (0..self.flag_variety.dim)
        ]

class IrreducibleEquivariantVectorBundle(EquivariantVectorBundle):
    def __init__(self, flag_variety, weight) -> None:
        self.weight = weight
        super().__init__(flag_variety, flag_variety.parabolic_subgroup.weight_muliplicities(weight))

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight}'

- Test of FlagVariety

In [218]:
X = FlagVariety(P)
X

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [219]:
X.dimension()

3

In [220]:
X.tangent_weights

[x0 - x2, x0 - x1, x0 - x3]

In [221]:
X.chern_classes()

[1,
 3*x0 - x1 - x2 - x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 x0^3 - x0^2*x1 - x0^2*x2 + x0*x1*x2 - x0^2*x3 + x0*x1*x3 + x0*x2*x3 - x1*x2*x3]

In [222]:
X.todd_classes()

[1,
 3/2*x0 - 1/2*x1 - 1/2*x2 - 1/2*x3,
 x0^2 - 2/3*x0*x1 + 1/12*x1^2 - 2/3*x0*x2 + 1/4*x1*x2 + 1/12*x2^2 - 2/3*x0*x3 + 1/4*x1*x3 + 1/4*x2*x3 + 1/12*x3^2,
 3/8*x0^3 - 3/8*x0^2*x1 + 1/12*x0*x1^2 - 3/8*x0^2*x2 + 7/24*x0*x1*x2 - 1/24*x1^2*x2 + 1/12*x0*x2^2 - 1/24*x1*x2^2 - 3/8*x0^2*x3 + 7/24*x0*x1*x3 - 1/24*x1^2*x3 + 7/24*x0*x2*x3 - 1/8*x1*x2*x3 - 1/24*x2^2*x3 + 1/12*x0*x3^2 - 1/24*x1*x3^2 - 1/24*x2*x3^2]

In [223]:
X.tangent_bundle()

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to {(1, 0, -1, 0): 1, (1, -1, 0, 0): 1, (1, 0, 0, -1): 1}

In [224]:
X.cotangent_bundle().chern_classes()

[1,
 -3*x0 + x1 + x2 + x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 -x0^3 + x0^2*x1 + x0^2*x2 - x0*x1*x2 + x0^2*x3 - x0*x1*x3 - x0*x2*x3 + x1*x2*x3]

In [225]:
[ f"{p} -> {chern_number(X, p)}"for p in Partitions(X.dimension())]

['[3] -> 4', '[2, 1] -> 24', '[1, 1, 1] -> 64']

- Test for EquivariantVectorBundle

In [226]:
w = (4, 0, 0)
E = IrreducibleEquivariantVectorBundle(X, w)
E

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (4, 0, 0)

In [227]:
E.base()

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [228]:
E.rank()

1

In [229]:
E.chern_classes()

[1, 4*x0, 0, 0]

In [230]:
E.chern_character()

[1, 4*x0, 8*x0^2, 32/3*x0^3]

In [231]:
E.todd_classes()

[1, 2*x0, 4/3*x0^2, 0]

In [232]:
class CompleteIntersection(IVariety):
    def __init__(self, flag_variety, vector_bundle) -> None:
        self.flag_variety = flag_variety
        self.vector_bundle = vector_bundle
        self.dim = self.flag_variety.dimension() - self.vector_bundle.rank()

    def __repr__(self) -> str:
        return f'a complete intersection of {self.flag_variety} and {self.vector_bundle}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        ci = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return ci
            def rank(self) -> int:
                return ci.dim
            def chern_classes(self) -> list:
                return ci.chern_classes()
        
        return VB()

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        def geometric_sequence(n, x):
            return sum( x^i for i in (0..n))
        
        cc = prod(1+x for x in self.flag_variety.tangent_weights) \
            * \
            prod(geometric_sequence(self.dim, -class_from_weight(vector(w)))^i 
                for w, i in self.vector_bundle.weight_muliplicities.items()
            )
        
        return [
            homogeneous_part(cc, i)
            for i in (0..self.dim)
        ]

    def numerical_integration_by_localization(self, f):
        top_of_f = homogeneous_part(f, self.dim)
        c_top    = self.vector_bundle.chern_classes()[self.vector_bundle.rank()] if self.dim >= 0 else 0
        return self.flag_variety.numerical_integration_by_localization(top_of_f * c_top)

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)


In [233]:
# K3
Y = CompleteIntersection(X, E) 
Y.dimension()

2

In [234]:
Y.chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [235]:
Y.tangent_bundle().chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [236]:
Y.cotangent_bundle().chern_classes()

[1,
 x0 + x1 + x2 + x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [237]:
Y.integration(Y.chern_classes()[Y.dimension()])

24

In [286]:
[ f"{p} -> {chern_number(Y, p)}"for p in Partitions(Y.dimension())]

['[2] -> 24', '[1, 1] -> 0']

Euler Characteristic
$$
\chi (X, E) = \int_X ch(E) td(X)
$$

In [238]:
def euler_characteristic(variety: IVariety, vector_bundle: IVectorBundle) -> int:
    chern_character = vector_bundle.chern_character()
    todd_classes    = variety.todd_classes()
    
    return variety.integration( 
        sum(
            chern_character[i]
            * todd_classes[variety.dimension() - i]
            for i in (0..variety.dimension())
        )
    )

In [239]:
euler_characteristic(X, E)

35

In [240]:
P.weyl_dimension(w)

35

- Tests for 
    - dual bundle
    - direct sum
    - tensor product
    - Symmetric products
    - Wedge products

In [241]:
E = IrreducibleEquivariantVectorBundle(X, (1, 1, 0))
E.rank()

3

In [242]:
dual_of_E = E.dual()
dual_of_E

the dual vector bundle of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0)

In [243]:
dual_of_E.rank()

3

In [244]:
E.chern_classes()

[1,
 6*x0 + x1 + x2 + x3,
 12*x0^2 + 4*x0*x1 + 4*x0*x2 + x1*x2 + 4*x0*x3 + x1*x3 + x2*x3,
 8*x0^3 + 4*x0^2*x1 + 4*x0^2*x2 + 2*x0*x1*x2 + 4*x0^2*x3 + 2*x0*x1*x3 + 2*x0*x2*x3 + x1*x2*x3]

In [245]:
dual_of_E.chern_classes()

[1,
 -6*x0 - x1 - x2 - x3,
 12*x0^2 + 4*x0*x1 + 4*x0*x2 + x1*x2 + 4*x0*x3 + x1*x3 + x2*x3,
 -8*x0^3 - 4*x0^2*x1 - 4*x0^2*x2 - 2*x0*x1*x2 - 4*x0^2*x3 - 2*x0*x1*x3 - 2*x0*x2*x3 - x1*x2*x3]

In [246]:
L1 = IrreducibleEquivariantVectorBundle(X, (2, 0, 0))
L2 = IrreducibleEquivariantVectorBundle(X, (5, 0, 0))
L3 = IrreducibleEquivariantVectorBundle(X, (7, 0, 0))

In [247]:
direct_sum(L1, L2)

the direct sum of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (2, 0, 0) and an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (5, 0, 0)

In [248]:
direct_sum(L1, L2).rank()

2

In [249]:
direct_sum(L1, L2).chern_classes()

[1, 7*x0, 10*x0^2, 0]

In [250]:
(L1 + L2 + L3).chern_classes()

[1, 14*x0, 59*x0^2, 70*x0^3]

In [251]:
E_otimes_L1 = tensor_product(E, L1)
E_otimes_L1

the tensor product of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0) and an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (2, 0, 0)

In [252]:
(L1 * L2 * L3).chern_classes()

[1, 14*x0]

In [253]:
def symmetric_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_symm_str_list = singular.chSymm(k, rank, l).sage_structured_str_list()
    ch_symm = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_symm_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_symm_str_list[0])
    base = vector_bundle.base()
    cc = [ 1 ] + [ cs(chern_classes) for cs in ch_symm ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th symmetric product of {vector_bundle}'

    return VB()

In [254]:
S3_of_E = symmetric_product(E, 3)
S3_of_E.rank()

10

In [255]:
S3_of_E.chern_classes()

[1,
 60*x0 + 10*x1 + 10*x2 + 10*x3,
 1620*x0^2 + 540*x0*x1 + 40*x1^2 + 540*x0*x2 + 95*x1*x2 + 40*x2^2 + 540*x0*x3 + 95*x1*x3 + 95*x2*x3 + 40*x3^2,
 25920*x0^3 + 12960*x0^2*x1 + 1920*x0*x1^2 + 82*x1^3 + 12960*x0^2*x2 + 4560*x0*x1*x2 + 357*x1^2*x2 + 1920*x0*x2^2 + 357*x1*x2^2 + 82*x2^3 + 12960*x0^2*x3 + 4560*x0*x1*x3 + 357*x1^2*x3 + 4560*x0*x2*x3 + 852*x1*x2*x3 + 357*x2^2*x3 + 1920*x0*x3^2 + 357*x1*x3^2 + 357*x2*x3^2 + 82*x3^3]

In [256]:
def wedge_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_wedge_str_list = singular.chWedge(k, rank, l).sage_structured_str_list()
    ch_wedge = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_wedge_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_wedge_str_list[0])
    base = vector_bundle.base()
    cc   = [ 1 ] + [ cs(chern_classes) for cs in ch_wedge ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th wedge product of {vector_bundle}'

    return VB()

In [257]:
L2_of_E = wedge_product(E, 2)
L2_of_E

the 2-th wedge product of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0)

In [258]:
L2_of_E.rank()

3

In [259]:
L2_of_E.chern_classes()

[1,
 12*x0 + 2*x1 + 2*x2 + 2*x3,
 48*x0^2 + 16*x0*x1 + x1^2 + 16*x0*x2 + 3*x1*x2 + x2^2 + 16*x0*x3 + 3*x1*x3 + 3*x2*x3 + x3^2,
 64*x0^3 + 32*x0^2*x1 + 4*x0*x1^2 + 32*x0^2*x2 + 12*x0*x1*x2 + x1^2*x2 + 4*x0*x2^2 + x1*x2^2 + 32*x0^2*x3 + 12*x0*x1*x3 + x1^2*x3 + 12*x0*x2*x3 + 2*x1*x2*x3 + x2^2*x3 + 4*x0*x3^2 + x1*x3^2 + x2*x3^2]

In [260]:
wedge_product(E, 3).rank()

1

In [261]:
wedge_product(E, 5).rank()

0

$\chi_y$-Genus
$$
\chi_y(X) = \int_X ch(\Lambda_y T^{\vee}_X) td(X)
$$

In [262]:
Q_y = QQ['y']
y = Q_y.gen()

def chi_y(variety: IVariety):
    cotangent_bundle = variety.cotangent_bundle()
    todd_classes = variety.todd_classes()

    coeff = [ 0 ] * (variety.dimension() + 1)

    for k in (0..variety.dimension()):
        chern_character = wedge_product(cotangent_bundle, k).chern_character()
        coeff[k] = variety.integration( 
            sum(
                chern_character[i]
                * todd_classes[variety.dimension() - i]
                for i in (0..variety.dimension())
            )
        )

    return sum(coeff[i] * y^i for i in (0..variety.dimension()))

In [263]:
chi_y(Y)

2*y^2 - 20*y + 2

In [264]:
chi_y(Y)(-1) # オイラー標数と同じ

24

- Elliptic Genus

$$
E_{q, y} = y^{-d/2} \bigotimes_{n \geq 1} 
\Lambda_{-yq^{n-1}} T^\vee_X 
\otimes 
\Lambda_{-y^{-1}q^n} T_X 
\otimes
Sym_{q^n} T^\vee_X
\otimes
Sym_{q^n} T_X
$$

$$
Ell_{q, y} = \int_X ch(E_{q, y})td(X)
$$

ここではプログラム処理上の理由で、$y^{d/2}Ell_{q,y}$を楕円種数と呼ぶことにする

In [265]:
import itertools

# 楕円種数のq^n*y^m次部分
def elliptic_genus_coefficient(variety: IVariety, n: int, m: int):
    cb = variety.cotangent_bundle()
    tb = variety.tangent_bundle()
    todd_classes = variety.todd_classes()

    wcb = [
        wedge_product(cb, i)
        for i in (0..max(variety.dimension(), n + 1))
    ]


    wtb = [
        wedge_product(tb, j)
        for j in (0..max(variety.dimension(), n))
    ]

    scb = [
        symmetric_product(cb, k)
        for k in (0..n)
    ]

    stb = [
        wedge_product(tb, l)
        for l in (0..n)
    ]

    indices = itertools.product(
        (0..max(variety.dimension(), n + 1)), 
        (0..max(variety.dimension(), n)), 
        (0..n), 
        (0..n)
    )
    indices = itertools.product(indices, repeat=n + 1)

    result = 0

    for x in indices:
        if sum(x[i][0] - x[i][1] for i in range(n + 1)) == m \
            and sum(x[i - 1][0] * (i - 1) + x[i - 1][1] * i + x[i - 1][2] * i + x[i - 1][3] * i for i in (1..n + 1)) == n:

            vb = prod(wcb[x[i][0]] * wtb[x[i][1]] * scb[x[i][2]] * stb[x[i][3]] for i in range(n + 1))
            chern_character = vb.chern_character()


            result += variety.integration(
                sum(
                    chern_character[i]
                    * todd_classes[variety.dimension() - i]
                    for i in (0..variety.dimension())
                )
            ) * (-1)^m

    return result

In [266]:
Q_qy.<q, y> = LaurentPolynomialRing(QQ)
# q, y = Q_qy.gens()

def elliptic_genus(variety: IVariety, cutoff_q: int, cutoff_y: int):
    result = 0
    for n in (0..cutoff_q):
        for m in (-cutoff_y..cutoff_y):
            result += elliptic_genus_coefficient(variety, n, m) * q^n * y^m

    return result

In [267]:
elliptic_genus(Y, 0, 3)

2*y^2 + 20*y + 2

In [268]:
elliptic_genus(Y, 1, 3) # 本当はこれにy^(-1)を掛けたものが正しい楕円種数

20*q*y^3 - 128*q*y^2 + 216*q*y + 2*y^2 - 128*q + 20*y + 20*q*y^-1 + 2

======= Quintic 3-fold =======

In [269]:
P = ParabolicSubgroup(CartanType('A4'), CartanType('A3'), [1])
P.dynkin_diagram()

X---O---O---O
1   2   3   4   
A4 with node 1 marked

In [270]:
X = FlagVariety(P)
E = IrreducibleEquivariantVectorBundle(X, (5, 0, 0, 0, 0))
quintic = CompleteIntersection(X, E)

In [271]:
quintic.chern_classes()

[1,
 -x0 - x1 - x2 - x3 - x4,
 11*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3 + 2*x0*x4 + x1*x4 + x2*x4 + x3*x4,
 -51*x0^3 - 13*x0^2*x1 - 13*x0^2*x2 - 3*x0*x1*x2 - 13*x0^2*x3 - 3*x0*x1*x3 - 3*x0*x2*x3 - x1*x2*x3 - 13*x0^2*x4 - 3*x0*x1*x4 - 3*x0*x2*x4 - x1*x2*x4 - 3*x0*x3*x4 - x1*x3*x4 - x2*x3*x4]

In [272]:
chi_y(quintic)

-100*y^2 + 100*y

In [273]:
elliptic_genus(quintic, 2, 3)

-800*q^2*y^3 + 2500*q^2*y^2 - 2300*q^2*y - 100*q*y^2 + 800*q^2 - 100*q*y - 100*y^2 + 100*q^2*y^-1 - 100*y + 100*q^2*y^-2 + 100*q*y^-1

In [287]:
[ f"{p} -> {chern_number(quintic, p)}"for p in Partitions(quintic.dimension())]

['[3] -> -200', '[2, 1] -> 0', '[1, 1, 1] -> 0']

======= G2 cases =======

In [274]:
G2 = CartanType('G2')
G2.dynkin_diagram()

  3
O=<=O
1   2   
G2

In [275]:
P1 = ParabolicSubgroup(G2, CartanType('A1'), [1])
P2 = ParabolicSubgroup(G2, CartanType('A1'), [2])

In [276]:
P1.dynkin_diagram()

  3
X=<=O
1   2   
G2 with node 1 marked

In [277]:
P2.dynkin_diagram()

  3
O=<=X
1   2   
G2 with node 2 marked

In [278]:
X1 = FlagVariety(P1)
X2 = FlagVariety(P2)

In [279]:
w = (1, 1)
E1 = IrreducibleEquivariantVectorBundle(X1, w)
E2 = IrreducibleEquivariantVectorBundle(X2, w)

In [280]:
Y1 = CompleteIntersection(X1, E1)
Y2 = CompleteIntersection(X2, E2)

In [281]:
Y1.chern_classes()

[1,
 0,
 3*x0^2 + x0*x1 - x1^2 - 7*x0*x2 + x1*x2 + 3*x2^2,
 -8*x0^3 + x0^2*x1 - x0*x1^2 + 23*x0^2*x2 + x1^2*x2 - 23*x0*x2^2 - x1*x2^2 + 8*x2^3]

In [282]:
Y2.chern_classes()

[1,
 0,
 5*x0^2 - 5*x0*x1 - x1^2 - 5*x0*x2 + 7*x1*x2 - x2^2,
 -14*x0^3 + 21*x0^2*x1 - 7*x0*x1^2 + 21*x0^2*x2 - 28*x0*x1*x2 + 7*x1^2*x2 - 7*x0*x2^2 + 7*x1*x2^2]

In [283]:
Y1.integration(Y1.chern_classes()[Y1.dimension()])

-98

In [284]:
Y2.integration(Y2.chern_classes()[Y2.dimension()])

-98

In [285]:
elliptic_genus(Y1, 2, 3)

KeyboardInterrupt: 

In [ ]:
elliptic_genus(Y2, 2, 3)

-392*q^2*y^3 + 1225*q^2*y^2 - 1127*q^2*y - 49*q*y^2 + 392*q^2 - 49*q*y - 49*y^2 + 49*q^2*y^-1 - 49*y + 49*q^2*y^-2 + 49*q*y^-1

======= F4 cases =======

In [288]:
G = CartanType('F4')
G.dynkin_diagram()

O---O=>=O---O
1   2   3   4   
F4

In [289]:
P1 = ParabolicSubgroup(G, CartanType('A1', 'A2'), [2])
P1

the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [290]:
P1.dynkin_diagram()

O---X=>=O---O
1   2   3   4   
F4 with node 2 marked

In [291]:
X1 = FlagVariety(P1)
X1

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [292]:
w1 = (0, 1, 1, 0)
E1 = IrreducibleEquivariantVectorBundle(X1, w1)
E1

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2] associated to (0, 1, 1, 0)

In [293]:
E1.rank()

3

In [294]:
# euler_characteristic(X1, E1)

In [295]:
P1.weyl_dimension(w1)

107406

In [296]:
w2 = (5, 0, 0, 0)
E2 = IrreducibleEquivariantVectorBundle(X1, w2)
E2.rank()

6

In [297]:
# euler_characteristic(X1, E2)

In [298]:
P1.weyl_dimension(w2)

627912

In [299]:
P2 = ParabolicSubgroup(G, CartanType('A2', 'A1'), [3])
P2

the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [300]:
P2.dynkin_diagram()

O---O=>=X---O
1   2   3   4   
F4 with node 3 marked

In [301]:
X2 = FlagVariety(P2)
X2

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [302]:
w3 = (0, 1, 1, 0)
E3 = IrreducibleEquivariantVectorBundle(X2, w3)
E3

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3] associated to (0, 1, 1, 0)

In [303]:
euler_characteristic(X2, E3)

107406

In [304]:
P2.weyl_dimension(w3)

107406

In [305]:
w4 = (2, 1, 0, 0)
E4 = IrreducibleEquivariantVectorBundle(X2, w4)
E4.rank()

15

In [306]:
euler_characteristic(X2, E4)

340119

In [307]:
P2.weyl_dimension(w4)

340119

In [308]:
Y1 = CompleteIntersection(X1, E1)
Y2 = CompleteIntersection(X2, E3)

In [309]:
# Y1.chern_classes()

In [310]:
# Y2.chern_classes()

In [319]:
# e1 = elliptic_genus(Y1, 1, 2)

Interrupting Singular...


KeyboardInterrupt: Restarting Singular (WARNING: all variables defined in previous session are now invalid)

IOStream.flush timed out


In [312]:
# e2 = elliptic_genus(Y2, 1, 2)

In [313]:
# e1 == e2

In [315]:
chern_number(Y1, [17])

-12566964323536824

In [316]:
chern_number(Y2, [17])

-12566964323536824

In [317]:
chern_number(Y1, [2] * 7 + [3])

-21029331652313088

In [318]:
chern_number(Y2, [2] * 7 + [3])

-28604369215531008

In [367]:
dim = 4
PolynomialRing(QQ, [f"c{i}" for i in (1..dim)], order=TermOrder('wdeglex', tuple(1..dim)))

Multivariate Polynomial Ring in c1, c2, c3, c4 over Rational Field

In [369]:
class DummyVariety(IVariety):
    def __init__(self, dim: int) -> None:
        self.dim = dim
        self.ring = PolynomialRing(QQ, [f"c{i}" for i in (1..dim)], order=TermOrder('wdeglex', tuple(1..dim)))

    def __repr__(self) -> str:
        return f"a dummy variety of dimension {self.dim}"

    def dimension(self) -> int:
        return self.dim

    def tangent_bundle(self) -> IVectorBundle:
        variety = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return variety
            
            def rank(self) -> int:
                return variety.dimension()
            
            def chern_classes(self) -> list:
                cs = variety.ring.gens()
                return [1] + list(cs)
        
        return VB()

    def integration(self, f) -> int:
        return homogeneous_part(f, self.dim)

In [370]:
d = DummyVariety(3)
d

a dummy variety of dimension 3

In [371]:
d.chern_classes()

[1, c1, c2, c3]

In [372]:
d.tangent_bundle().chern_character()

[3, c1, 1/2*c1^2 - c2, 1/6*c1^3 - 1/2*c1*c2 + 1/2*c3]

In [375]:
[ f"{p} -> {chern_number(d, p)}"for p in Partitions(d.dimension())]

['[3] -> c3', '[2, 1] -> c1*c2', '[1, 1, 1] -> c1^3']